In [1]:
import torch 
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [19]:
n_embedd  = 384 #number of total dims we want to capture from all heads concatenated together
n_decoders = 4  #4 decoder layers
dropout = 0.2 #20% neurons are dropped out to prevent overfitting
num_attention_heads= 4 # number of attention heads running in parallel
max_iters = 1000  #number of training itterations
learning_rate = 3e-3 #for optimizer
eval_iters = 100 #for reporting loss
block_size = 64  #size of one input sequence
batch_size = 128  #how many block sized input sequences we want at the same time

In [20]:
chars = ""   #open web text - cant read 45 gbs of text at once 
with open('data\Two sailor lads.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()
    chars = sorted(set(text))
print(chars, len(chars))

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '3', '5', '7', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '£', '§', 'Ñ', 'æ', 'é', 'ê', 'ñ', 'ö', '†', '‡', '\ufeff'] 86


In [21]:
##vocab
text_to_int = {char:idx for idx, char in enumerate(chars)}
int_to_text = {idx:char for idx, char in enumerate(chars)}

In [22]:
encode = lambda z: [text_to_int[char] for char in z]
decode = lambda z: ''.join(int_to_text[int] for int in z)

In [23]:
encoded = encode('hello')
encoded

[56, 53, 60, 60, 63]

In [24]:
decode(encoded)

'hello'

In [25]:
data = torch.tensor(encode(text), dtype = torch.long) #long seq of integers

In [26]:
#validation and training split
split_index = int(0.8 * len(text))
train = data[:split_index]
test = data[split_index:]
print(len(train), len(test))

380708 95178


In [27]:
block_size = 8
batch_size = 4
def get_batched_data(split):
    data = train if 'train' else 'test'
    random_indices = torch.randint((len(data)-block_size), (batch_size,)) #random indices of batch size inside the data
    #print(random_indices)
    x = torch.stack([data[i:i+block_size]for i in random_indices]) #stacking a batch of 4 input block each of block size = 8 
    y = torch.stack([data[i+1:i+block_size+1]for i in random_indices]) #stacking a batch of 4 target block each of block size = 8 
    return x.to(device), y.to(device)

X,y = get_batched_data('train')
print(f'inputs: {X}, outputs: {y}')

inputs: tensor([[ 1, 51, 49, 62,  1, 67, 68, 53],
        [ 1, 60, 53, 54, 68,  1, 57, 68],
        [69, 68, 57, 63, 62,  1, 68, 63],
        [63, 71, 62,  1, 20, 61, 53, 66]], device='cuda:0'), outputs: tensor([[51, 49, 62,  1, 67, 68, 53, 53],
        [60, 53, 54, 68,  1, 57, 68,  1],
        [68, 57, 63, 62,  1, 68, 63,  1],
        [71, 62,  1, 20, 61, 53, 66, 57]], device='cuda:0')


In [28]:
#scaled dot product attention

#each head has 96 features and 4 heads in parallel = 384 features

class AttentionHead(nn.Module):

    def __init__(self, head_size):  # head_size-> n_embedd // num_attention_heads  384//4 -> 96
        super().__init__()
        self.key = nn.Linear(n_embedd, head_size, bias = False)  #384 -> 96 features
        self.query = nn.Linear(n_embedd, head_size, bias = False)
        self.value = nn.Linear(n_embedd, head_size, bias = False)
        #register buffer registers no look ahead masking in the model state
        #instaed of reinitializing it for every single head for every single forward and backward pass we add it to model state
        #saves computation
        #efficient way- reduces training time
        #training can be donw without this but it would take longer to complete
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))#(8,8)
        self.dropout = nn.Dropout(dropout) #20% neurons are dropped out

    def forward(self, x):
        #xshape: (B,T,C) --> (4,8,384)
        B,T,C = x.shape  #(4,8,384)
        k = self.key(x) # C or n_embedd (384) -> head_size(96)  (B,T,C) -> (B,T,hs)
        q = self.query(x) # (B,T,C= 384) -> (B,T,hs = 96)
        #calculating attention scores
        #key.transpose(-2,-1) - interchanging last two dims (B,hs,T) = (4, 96, 8)
        #SCALING: *key.shape[-1] = 96 -> sqrt(96) for 
        #query(B,T,hs) @ key(B,hs,T) -> (B, T, T)
        scores = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        #masked fill - repacing 0 to -inf for T timestamp
        #masked fill to prevent look ahead
        scores = scores.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        scores = F.softmax(scores, dim = -1) #(B,T,T) #confidence scores for prediction
        scores = self.dropout(scores)  
        v = self.value(x)  #(B,T,hs)
        out = scores @ v
        #print(f'shape after attention head: {out.shape}')
        return out

In [29]:
#keys and queries part will be implemented during dot product attention 

class MultiHeadAttention(nn.Module):
    def __init__(self, num_attention_heads, head_size): ##(4, 96)
        super().__init__()
        #bunch of heads in parallel for each head
        self.multiheads = nn.ModuleList([AttentionHead(head_size) for _ in range(num_attention_heads)]) #4 heads running in parallel
        #below transformation adds another set of learnable parameters (weights and bias)
        #kinda helps network learn more about the the text fed
        self.proj = nn.Linear(head_size * num_attention_heads, n_embedd)  #head_size(96) * num_attention_head(4) = 384 ---projected_to---> n_embedd(384)
        self.dropout = nn.Dropout(dropout)  #dropping out 20% of neurons

    def forward(self, x):
        out = torch.cat([head(x) for head in self.multiheads], dim = -1) #concatenation the out each head along the last dimension  (B,T,C)-> concatenate along channel dim
        #4 heads running in parallel each having output (B,T,hs) 
        # on concatenating across dim=-1 output shape = (B,T, 4*hs) -> (B,T,C) 
        out = self.dropout(self.proj(out))
        #print(f'out shape after multi head attention mech: {out.shape}')
        return out 

In [30]:
class FeedForward(nn.Module):
    def  __init__(self, n_embedd):
        super().__init__()
        self.netw = nn.Sequential(
            nn.Linear(n_embedd, 4 * n_embedd),
            nn.ReLU(),
            nn.Linear(4 * n_embedd, n_embedd),
            nn.Dropout(dropout),
        )
    def forward(self,x):
        return self.netw(x)

In [31]:
class Decoder_block(nn.Module):

    def __init__(self, n_embedd, num_attention_heads):  #input shape to decoder block -> (B, T, C)-(4,8,384)
        super().__init__()
        #n_head is the number of heads and embedd is the embedding dim
        #each head captures 96 features -> head size
        #4 attentions heads working in parallel
        #how many features are each of them capturing?
        head_size = n_embedd // num_attention_heads #is the number of features that each head will be capturing-> 384 // 4 - > 96 head_size
        self.sa = MultiHeadAttention(num_attention_heads, head_size)  #(4, 96)
        self.ffwd = FeedForward(n_embedd) #(384)
        self.ln1 = nn.LayerNorm(n_embedd) #(384) for add and norm
        self.ln2 = nn.LayerNorm(n_embedd) #(384) 

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)  #post norm arch -  Add and Norm. #skip connection 
        y = self.ffwd(x)
        x = self.ln2(x + y)  #residual
        #print(f'shape after Decode block: {x.shape}')
        return x

In [32]:
class GPTmodel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        #what is embedding matrix
        self.embedding_matrix = nn.Embedding(vocab_size, n_embedd)  #char level encoding art to vocab
        self.positional_matrix = nn.Embedding(block_size, n_embedd)  #char level positional embedding wrt to position of each char in a block
        
        self.decoder_blocks = nn.Sequential(*[Decoder_block(n_embedd, num_attention_heads  = num_attention_heads) for _ in range(n_decoders)])
        
        self.final_layer_norm = nn.LayerNorm(n_embedd)
        self.linear_head_final = nn.Linear(n_embedd, vocab_size)   #layer to get the next character given the input characters in hte vocab later softmax will be used to get the most probable character 
        
        #initializing weights with mean = 0.0 and  std dev = 0.02
        #helps training coverge better
        #weights are initialized properly
        self.apply(self._init_weights)

    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
                  

    
    def forward(self, index, targets = None):
        #print(index.shape)
        B, T = index.shape

        
        token_embeddings = self.embedding_matrix(index)  ##index (4,8) (B,T) --token_embeddings--> (4,8,384)(B,T,C)
        pos_embeddings = self.positional_matrix(torch.arange(T, device = device))
        x = token_embeddings + pos_embeddings  #broadcasting helps- check broadcasting semantics in torch SHAPE - (4,8,384)
        #print(x.shape)
        x = self.decoder_blocks(x)
        x = self.final_layer_norm(x)
        #print(f'final output shape before linear layer: {x.shape}')
        logits = self.linear_head_final(x)
        #print(f'final output shape: {logits.shape}')

        
        #what is logits?
        #logits = self.embedding_matrix(index)
        if targets is None:
            print('FLAG')
            loss = None
        else:          
            
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            #print(f'shape after reshaping logits: {logits.shape} and targets shape: {targets.shape}')
            loss = F.cross_entropy(logits, targets)
            #print(loss.item())
            
        return logits, loss

    def generate(self,index,max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:,-1,:]  #dim ain't dimensioning: it is now as targets are default to None so no loss in logits dim(3-dim)
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples = 1)
            index = torch.cat((index, index_next), dim = 1)
        return index

model = GPTmodel(vocab_size = len(chars))
m = model.to(device)

# context = torch.zeros((1,1), dtype = torch.long, device= device)
# generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
# print(generated_chars)

In [33]:
@torch.no_grad()  #decorator for no grad since we are only caLculating the loss we do not need to compute gradient as no weight updation (optimization) happens here
def estimate_loss():
    out = {} 
    model.eval()  #eval model for model evaluation
    for split in ['train', 'test']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batched_data(split)
            logits, loss = model(X, Y)  #model pass without using any gradients
            losses[k] = loss.item()
        out[split] = losses.mean()  #averaging loss over iters
    model.train()  #puts model into training model: weights and biases are updated during this mode
    return out

In [34]:
# ##creating optimizer 
# max_iters = 1000
# learning_rate = 3e-4
# eval_iters = 250  #every 250 iterations evaluate the model

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        out = estimate_loss()
        print(f"Step: {iter}, Training loss: {out['train']:.4f}, Evaluation loss: {out['test']:.4f}")
    #sample from batch of data
    batched_x, batched_y = get_batched_data(split = 'train')
    #evaluating loss and performing back propagation
    logits, loss = m.forward(batched_x, batched_y)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()  #differentiate the loss fn wrt parameters
    optimizer.step() #update the parameters
    
print(loss.item()) #.item() get value from torch tensor

Step: 0, Training loss: 4.5514, Evaluation loss: 4.5513
Step: 100, Training loss: 3.2692, Evaluation loss: 3.2543
Step: 200, Training loss: 3.2315, Evaluation loss: 3.2261
Step: 300, Training loss: 3.1777, Evaluation loss: 3.0994
Step: 400, Training loss: 3.1762, Evaluation loss: 3.1713
Step: 500, Training loss: 3.1807, Evaluation loss: 3.2158
Step: 600, Training loss: 3.1827, Evaluation loss: 3.1536
Step: 700, Training loss: 3.1817, Evaluation loss: 3.1843
Step: 800, Training loss: 3.1024, Evaluation loss: 3.1582
Step: 900, Training loss: 3.0991, Evaluation loss: 3.1446
3.1766104698181152
